<a href="https://colab.research.google.com/github/rebuyguy2000/ex-date-payday-public/blob/main/yahoo_fin_dividend_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yahoo_fin

In [ ]:
import pandas as pd 
from pandas_datareader import DataReader
from yahoo_fin import stock_info as si
from yahoo_fin.stock_info import get_data
import requests
import datetime
from datetime import timedelta
from datetime import date
import csv

In [ ]:
# initialize dataframes
# a_hist appended version of hist
# hist used to hold and calculate price history
# sec_list used for all securities to be analyzed

a_hist = pd.DataFrame()
hist = pd.DataFrame()
sec_list = pd.DataFrame()

# initialize variables
rec_no = 0
ticker = ""


In [ ]:
# open csv of monthly dividend paying securities

rec_no = 20
sec_list = pd.read_csv('actual_trades_2021.csv')
sec_list.sort_values(by=['ex-date'])
ticker = sec_list.iloc[rec_no,0]

print(sec_list.iloc[rec_no])
#sec_list.info()

symbol             PZN
ex-date     2021-02-11
pay date    2021-02-25
div               0.25
Name: 20, dtype: object


In [ ]:
# use timedelta function to set day interval back 12 days
# get ex-date from sec_list dataframe
# set new_start_d to ex-date - interval

ten_day = timedelta(days=-15)
start_d = sec_list.iloc[rec_no,1]
temp_d = date.fromisoformat(start_d)
temp_d = temp_d + ten_day
new_start_d = date.isoformat(temp_d)

print(new_start_d)

# change end_d so that it includes the ex-date
# add one more day to end_d so that the yfinance get_data includes the ex-date

one_day = timedelta(days=1)
start_d = sec_list.iloc[rec_no,1]
temp_d = date.fromisoformat(start_d)
temp_d = temp_d + one_day
new_end_d = date.isoformat(temp_d)

print(new_end_d)


2021-01-27
2021-02-12


In [ ]:
# use symbol from sec_list to retrieve past history
# use new_start_day to start 10 days earlier than ex-date
# ticker initialized above when reading csv and choosing record number

intvl = "1d"

hist = get_data(ticker, start_date = new_start_d, end_date = new_end_d, index_as_date = False, interval = intvl)

gain = hist['open']
hist.insert(loc=1, column='gains', value=gain)

hist


,date,gains,open,high,low,close,adjclose,volume,ticker
0,2021-01-27,8.23,8.23,8.48,8.05,8.25,8.038462,91100,PZN
1,2021-01-28,8.38,8.38,8.53,8.24,8.43,8.213846,47200,PZN
2,2021-01-29,8.41,8.41,8.41,8.06,8.30,8.087180,49900,PZN
3,2021-02-01,8.33,8.33,8.47,8.19,8.41,8.194359,54800,PZN
4,2021-02-02,8.54,8.54,8.72,8.31,8.72,8.496410,49900,PZN
5,2021-02-03,8.57,8.57,8.76,8.40,8.59,8.369743,52200,PZN
6,2021-02-04,8.61,8.61,8.91,8.50,8.84,8.613334,76300,PZN
7,2021-02-05,8.92,8.92,9.18,8.84,9.18,8.944615,62800,PZN
8,2021-02-08,9.35,9.35,9.56,9.23,9.56,9.314872,85500,PZN
9,2021-02-09,9.69,9.69,9.91,9.61,9.80,9.548718,84700,PZN


In [ ]:
x = 0
y = hist.shape[0]

while x < hist.shape[0]:
    hist.iloc[x,1] = hist.iloc[y-2,2] - hist.iloc[x,2]
    print("Updating frame, on %d now!" % (x))
    #print(hist.iloc[x+1,0])
    #print(hist.iloc[x,1])
    x += 1

hist

Updating frame, on 0 now!
Updating frame, on 1 now!
Updating frame, on 2 now!
Updating frame, on 3 now!
Updating frame, on 4 now!
Updating frame, on 5 now!
Updating frame, on 6 now!
Updating frame, on 7 now!
Updating frame, on 8 now!
Updating frame, on 9 now!
Updating frame, on 10 now!
Updating frame, on 11 now!


,date,gains,open,high,low,close,adjclose,volume,ticker
0,2021-01-27,1.570001,8.23,8.48,8.05,8.25,8.038462,91100,PZN
1,2021-01-28,1.420000,8.38,8.53,8.24,8.43,8.213846,47200,PZN
2,2021-01-29,1.390000,8.41,8.41,8.06,8.30,8.087180,49900,PZN
3,2021-02-01,1.470000,8.33,8.47,8.19,8.41,8.194359,54800,PZN
4,2021-02-02,1.260000,8.54,8.72,8.31,8.72,8.496410,49900,PZN
5,2021-02-03,1.230000,8.57,8.76,8.40,8.59,8.369743,52200,PZN
6,2021-02-04,1.190001,8.61,8.91,8.50,8.84,8.613334,76300,PZN
7,2021-02-05,0.880000,8.92,9.18,8.84,9.18,8.944615,62800,PZN
8,2021-02-08,0.450000,9.35,9.56,9.23,9.56,9.314872,85500,PZN
9,2021-02-09,0.110001,9.69,9.91,9.61,9.80,9.548718,84700,PZN


In [ ]:
# append calculated gain column along with 
ten_row = hist.iloc[0]
nine_row = hist.iloc[1]

#append row to the dataframe
a_hist = a_hist.append(ten_row, ignore_index=True)
a_hist = a_hist.append(nine_row, ignore_index=True)

a_hist

,adjclose,close,date,gains,high,low,open,ticker,volume
0,32.388870,32.810001,2021-02-08,0.949997,32.810001,32.332001,32.400002,NTB,129700.0
1,32.813351,33.240002,2021-02-09,0.829998,33.270000,32.380001,32.520000,NTB,103600.0
2,7.021340,7.200000,2021-02-08,1.300000,7.460000,6.620000,6.620000,NHTC,75100.0
3,7.079851,7.260000,2021-02-09,0.620000,7.310000,6.980000,7.300000,NHTC,46700.0
4,51.776897,53.169998,2021-02-10,3.599998,54.540001,51.000000,54.540001,ALTM,54600.0
5,51.708733,53.099998,2021-02-11,5.130001,54.380001,51.689999,53.009998,ALTM,30700.0
6,20.627205,20.820000,2021-03-01,1.059999,20.889999,20.330000,20.330000,RA,479000.0
7,20.706463,20.900000,2021-03-02,0.500000,21.290001,20.674999,20.889999,RA,526600.0
8,8.987920,9.300000,2021-02-16,-0.389999,9.440000,9.020000,9.230000,FLNG,105200.0
9,9.132886,9.450000,2021-02-17,-0.410000,9.550000,9.250000,9.250000,FLNG,58400.0


In [ ]:
#a_hist.info()
hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      11 non-null     datetime64[ns]
 1   gains     11 non-null     float64       
 2   open      11 non-null     float64       
 3   high      11 non-null     float64       
 4   low       11 non-null     float64       
 5   close     11 non-null     float64       
 6   adjclose  11 non-null     float64       
 7   volume    11 non-null     int64         
 8   ticker    11 non-null     object        
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 920.0+ bytes


In [ ]:
# write file to colab drive directory (folder icon on left side bar)

hist.to_csv('hist.csv',index=False)